## Cleaning the Raw Blockbusters_2019_1977 CSV File for Exploratory Data Analysis ##

In [1]:
# Import Dependencies
import pandas as pd
import numpy as np
import sqlite3

In [2]:
# Load Blockbusters_2019_1977 CSV Into DataFrame, Encoding Set To 'latin-1' To Handle Special Characters
movie_data = pd.read_csv("Resources/Blockbusters_2019_1977.csv", encoding='latin-1')

In [3]:
# Display Information About movie_data DataFrame
movie_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 430 entries, 0 to 429
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   release_year          430 non-null    int64  
 1   rank_in_year          430 non-null    int64  
 2   imdb_rating           430 non-null    float64
 3   mpaa_rating           429 non-null    object 
 4   film_title            430 non-null    object 
 5   film_budget           430 non-null    object 
 6   length_in_min         430 non-null    int64  
 7   domestic_distributor  430 non-null    object 
 8   worldwide_gross       430 non-null    object 
 9   domestic_gross        430 non-null    object 
 10  genre_1               430 non-null    object 
 11  genre_2               409 non-null    object 
 12  genre_3               324 non-null    object 
dtypes: float64(1), int64(3), object(9)
memory usage: 43.8+ KB


In [4]:
# Remove Commas From Selected Columns To Convert From Strings To Numeric Values
movie_data[['film_budget','worldwide_gross','domestic_gross']] = movie_data[
    ['film_budget', 'worldwide_gross', 'domestic_gross']]\
        .apply(lambda x: x.str.replace(',', ''))

# Convert 'film_budget', 'worldwide_gross', 'domestic_gross' Data Types
movie_data['film_budget'] = movie_data['film_budget'].astype(int)
movie_data['worldwide_gross'] = movie_data['worldwide_gross'].astype(float)
movie_data['domestic_gross'] = movie_data['domestic_gross'].astype(float)

# Replace Empty Strings With NaN Values In Entire DataFrame
movie_data.replace('', np.nan, inplace=True)

# Apply The strip() Function To Remove Leading And Trailing Whitespaces From Selected Columns
movie_data[['mpaa_rating',
            'film_title',
            'domestic_distributor',
            'genre_1',
            'genre_2',
            'genre_3']] = movie_data[[
    'mpaa_rating',
    'film_title',
    'domestic_distributor',
    'genre_1',
    'genre_2',
    'genre_3']].apply(lambda x: x.str.strip())

# Replace Specific Values In The 'genre_2' & 'genre_3' Columns With Their Corresponding Replacements
movie_data['genre_2'] = movie_data['genre_2'].replace({'Music':'Musical','romance':'Romance'})
movie_data['genre_3'] = movie_data['genre_3'].replace({'Music':'Musical'})

# Calculate Profit By Subtracting Film Budget From Worldwide/Domestic Gross
movie_data['worldwide_profit'] = movie_data['worldwide_gross'] - movie_data['film_budget']
movie_data['domestic_profit'] = movie_data['domestic_gross'] - movie_data['film_budget']

# Reorganize The DataFrame
movie_data = movie_data[['film_title',
                         'genre_1',
                         'genre_2',
                         'genre_3',
                         'release_year',
                         'domestic_distributor',
                         'mpaa_rating',
                         'length_in_min',
                         'imdb_rating',
                         'film_budget',
                         'domestic_gross',
                         'domestic_profit',
                         'worldwide_gross',
                         'worldwide_profit',
                         'rank_in_year'
                         ]]

# Rename 'rank_in_year' Column To 'rank_year_ww_gross'
movie_data.rename(columns={'rank_in_year':'rank_year_ww_gross'}, inplace=True)

In [5]:
# Display Cleaned DataFrame
movie_data.head()

,film_title,genre_1,genre_2,genre_3,release_year,domestic_distributor,mpaa_rating,length_in_min,imdb_rating,film_budget,domestic_gross,domestic_profit,worldwide_gross,worldwide_profit,rank_year_ww_gross
0,Avengers: Endgame,Action,Adventure,Drama,2019,Walt Disney,PG-13,181,8.5,356000000,858373000.0,502373000.0,2.797801e+09,2.441801e+09,1
1,The Lion King,Animation,Adventure,Drama,2019,Walt Disney,PG,118,7.0,260000000,543638043.0,283638043.0,1.656943e+09,1.396943e+09,2
2,Frozen II,Animation,Adventure,Comedy,2019,Walt Disney,PG,103,7.2,150000000,470089732.0,320089732.0,1.420560e+09,1.270560e+09,3
3,Spider-Man: Far from Home,Action,Adventure,Sci-Fi,2019,Sony Pictures,PG-13,129,7.6,160000000,390532085.0,230532085.0,1.131928e+09,9.719280e+08,4
4,Captain Marvel,Action,Adventure,Sci-Fi,2019,Walt Disney,PG-13,123,6.9,175000000,426829839.0,251829839.0,1.128275e+09,9.532748e+08,5


In [6]:
# Connect To SQLite Database (Create If Doesn't Exist)
conn = sqlite3.connect('Resources/Blockbusters_2019_1977.db')

# Write DataFrame To SQLite Database Table
movie_data.to_sql('movie_data', conn, if_exists='replace', index=False)

# Close Database Connection
conn.close()

#### Further Cleaning and Reorgnization for Hover Data Associated with Scatter, Bar, and Pie Graphs ####

In [7]:
# Create New DataFrame For Scatter/Bar/Pie Graphs
graph_movie_data = movie_data.copy()

# Combine Genre Columns Into Single Column 'genres'
graph_movie_data['genres'] = graph_movie_data[['genre_1', 'genre_2', 'genre_3']]\
    .apply(lambda x: ', '.join(x.dropna()), axis=1)

# Reorganize The DataFrame
graph_movie_data = graph_movie_data[['film_title',
                         'genre_1',
                         'genre_2',
                         'genre_3',
                         'genres',
                         'release_year',
                         'domestic_distributor',
                         'mpaa_rating',
                         'length_in_min',
                         'imdb_rating',
                         'film_budget',
                         'domestic_gross',
                         'domestic_profit',
                         'worldwide_gross',
                         'worldwide_profit',
                         'rank_year_ww_gross'
                         ]]

# Display DataFrame
graph_movie_data.head()

,film_title,genre_1,genre_2,genre_3,genres,release_year,domestic_distributor,mpaa_rating,length_in_min,imdb_rating,film_budget,domestic_gross,domestic_profit,worldwide_gross,worldwide_profit,rank_year_ww_gross
0,Avengers: Endgame,Action,Adventure,Drama,"Action, Adventure, Drama",2019,Walt Disney,PG-13,181,8.5,356000000,858373000.0,502373000.0,2.797801e+09,2.441801e+09,1
1,The Lion King,Animation,Adventure,Drama,"Animation, Adventure, Drama",2019,Walt Disney,PG,118,7.0,260000000,543638043.0,283638043.0,1.656943e+09,1.396943e+09,2
2,Frozen II,Animation,Adventure,Comedy,"Animation, Adventure, Comedy",2019,Walt Disney,PG,103,7.2,150000000,470089732.0,320089732.0,1.420560e+09,1.270560e+09,3
3,Spider-Man: Far from Home,Action,Adventure,Sci-Fi,"Action, Adventure, Sci-Fi",2019,Sony Pictures,PG-13,129,7.6,160000000,390532085.0,230532085.0,1.131928e+09,9.719280e+08,4
4,Captain Marvel,Action,Adventure,Sci-Fi,"Action, Adventure, Sci-Fi",2019,Walt Disney,PG-13,123,6.9,175000000,426829839.0,251829839.0,1.128275e+09,9.532748e+08,5


In [8]:
# Connect To SQLite Database (Create If Doesn't Exist)
conn = sqlite3.connect('Resources/scatter_bar_pie.db')

# Write DataFrame To SQLite Database Table
graph_movie_data.to_sql('graph_movie_data', conn, if_exists='replace', index=False)

# Close Database Connection
conn.close()